In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import yaml
from copy import deepcopy

from n3jet.utils import FKSPartition
from n3jet.utils.general_utils import (
    bool_convert,
    dot
)

In [36]:
# original NJet to compare against
nj_compare_dir = '/mt/batch/jbullock/Sherpa_NJet/runs/diphoton/4g2A/RAMBO/parallel_fixed/integration_grid/NJet_NJet_unit_grid_2/'
hepmc_mom_file_nj_compare = nj_compare_dir + 'full_momenta_events_1.5M_new_sherpa_cuts_PDF.npy'
events_file_nj_compare = nj_compare_dir + 'original_weights_events_1.5M_new_sherpa_cuts_PDF.npy'
trials_file_nj_compare = nj_compare_dir + 'trials_events_1.5M_new_sherpa_cuts_PDF.npy'

# NN files for reweighting
nn_dir = '/mt/batch/jbullock/Sherpa_NJet/runs/diphoton/4g2A/RAMBO/parallel_fixed/integration_grid/nn_NJet_unit_grid_2_delta_0001/'
reweight_raw_dir = nn_dir + 'reweight_raw/'
reweigth_points = nn_dir + 'reweight_near_05.npy'
mom_file_nn = nn_dir + 'unshuffle_momenta_events_1.5M_new_sherpa_cuts_PDF.npy'
me_file_nn = nn_dir + 'unshuffle_events_NN_1.5M_new_sherpa_cuts_PDF_loop.npy'
hepmc_mom_file_nn = nn_dir + 'full_momenta_events_1.5M_new_sherpa_cuts_PDF.npy'
events_file_nn = nn_dir + 'original_weights_events_1.5M_new_sherpa_cuts_PDF.npy'
trials_file_nn = nn_dir + 'trials_events_1.5M_new_sherpa_cuts_PDF.npy'

In [12]:
all_legs = True
delta_cut = 0.0
delta_near = 0.001

In [34]:
def len_check(x):
    if x is None:
        raise ValueError('Value is None')
    elif len(x) > 1:
        raise ValueError('length of array is > 1')
    elif len(x) < 1:
        raise ValueError('length of array is < 1')
    else:
        return x[0]
    
def reconcile(to_match, match_to):
    indices = []
    for idx, i in tqdm(enumerate(to_match)):
        try_all = np.where(np.all(match_to==i, axis=(1, 2)))[0]
        if len(try_all) == 0:
            try_2 = np.where(np.all(match_to[:,2]==i[2], axis=(1)))[0]
            if len(try_2) == 0:
                try_3 = np.where(np.all(match_to[:,3]==i[3], axis=(1)))[0]
                if len(try_3) == 0:
                    try_4 = np.where(np.all(match_to[:,4]==i[4], axis=(1)))[0]
                    if len(try_4) == 0:
                        print ('Struggling on index: {}'.format(idx))
                        indices.append(None)
                    else:
                        indices.append(try_4)
                else:
                    indices.append(try_3)
            else:
                indices.append(try_2)
        else:
            indices.append(try_all)
    return indices

def index_process(x):
    indices = []
    for i in tqdm(x):
        index = len_check(i)
        indices.append(index)
        
    return indices

## Load data

In [13]:
chosen_max = np.load(reweigth_points)

In [15]:
reweight_data = []
for i in os.listdir(reweight_raw_dir):
    if i.split(".")[-1] == "part":
        with open(reweight_raw_dir + i, 'r') as f:
            reweight_data.append(float(f.read().split(" ")[0]))

In [16]:
assert len(chosen_max) == len(reweight_data)

In [37]:
hepmc_mom_nj_compare = np.load(hepmc_mom_file_nj_compare, allow_pickle=True)
events_nj_compare = np.load(events_file_nj_compare, allow_pickle=True)
trials_nj_compare = np.load(trials_file_nj_compare, allow_pickle=True)

mom_nn = np.load(mom_file_nn, allow_pickle=True)
me_nn = np.load(me_file_nn, allow_pickle=True)
hepmc_mom_nn = np.load(hepmc_mom_file_nn, allow_pickle=True)
events_nn = np.load(events_file_nn, allow_pickle=True)
trials_nn = np.load(trials_file_nn, allow_pickle=True)

## Cross-section check

In [21]:
xs_compare = np.sum(events_nj_compare)/np.sum(trials_nj_compare)
print ("Comparison XS is {}".format(xs_compare))

Comparison XS is 4.18227928531e-06


In [33]:
nn_xs = np.sum(events_nn)/np.sum(trials_nn)
print ("NN XS is {}".format(nn_xs)) 

NN XS is 4.38744671886e-06


## Split data

In [18]:
nn_fks_partition = FKSPartition(
    momenta = list(hepmc_mom_nn),
    labels = events_nn,
    all_legs = all_legs
)

nn_cut_momenta, nn_near_momenta, nn_cut_labels, nn_near_labels = nn_fks_partition.cut_near_split(
    delta_cut = delta_cut,
    delta_near = delta_near
)

100%|██████████| 1500000/1500000 [02:05<00:00, 11978.69it/s]


In [38]:
nn_me_fks_partition = FKSPartition(
    momenta = list(mom_nn),
    labels = me_nn,
    all_legs = all_legs
)

nn_me_cut_momenta, nn_me_near_momenta, nn_me_cut_labels, nn_me_near_labels = nn_me_fks_partition.cut_near_split(
    delta_cut = delta_cut,
    delta_near = delta_near
)


 18%|█▊        | 275674/1500340 [00:13<00:58, 20888.52it/s]


 37%|███▋      | 552891/1500340 [00:26<00:45, 21031.06it/s]


 55%|█████▌    | 827940/1500340 [00:39<00:31, 21309.76it/s]


 73%|███████▎  | 1101812/1500340 [00:52<00:19, 20442.47it/s]


 91%|█████████▏| 1372544/1500340 [01:05<00:06, 21182.09it/s]


100%|██████████| 1500340/1500340 [01:11<00:00, 20928.60it/s]

In [39]:
nn_near_round = np.round(nn_near_momenta, 6)
nn_me_near_round = np.round(nn_me_near_momenta, 6)

## Reweight

In [40]:
reweight_props = np.arange(0,0.6,0.1)

In [42]:
nn_near_chosen = nn_near_round[chosen_max]

In [43]:
nn_near_chosen[0]

array([[ 183.173961,    0.      ,    0.      ,  183.173961],
       [ 170.022834,    0.      ,    0.      , -170.022834],
       [ 128.988407,  -20.091576,  102.495194,   75.690639],
       [ 114.587362,  -20.016646, -107.04783 ,   35.642101],
       [ 107.445019,   41.714001,    3.308502,  -98.961751],
       [   2.176007,   -1.605778,    1.244134,    0.780137]])

In [45]:
nn_me_indices = reconcile(nn_near_chosen, nn_me_near_round)


0it [00:00, ?it/s]
35it [00:00, 343.20it/s]
81it [00:00, 370.75it/s]
132it [00:00, 402.87it/s]
189it [00:00, 441.17it/s]
248it [00:00, 477.19it/s]
308it [00:00, 506.35it/s]
368it [00:00, 529.73it/s]
420it [00:00, 525.89it/s]
483it [00:00, 552.75it/s]
552it [00:01, 585.55it/s]
620it [00:01, 610.22it/s]
682it [00:01, 589.23it/s]
742it [00:01, 538.97it/s]
799it [00:01, 546.85it/s]
857it [00:01, 556.03it/s]
916it [00:01, 563.34it/s]
973it [00:01, 555.53it/s]
1029it [00:01, 539.26it/s]
1084it [00:01, 527.42it/s]
1139it [00:02, 533.06it/s]
1206it [00:02, 567.16it/s]
1273it [00:02, 594.01it/s]
1340it [00:02, 613.71it/s]
1407it [00:02, 627.29it/s]
1485it [00:02, 666.38it/s]
1567it [00:02, 704.85it/s]
1645it [00:02, 725.24it/s]
1719it [00:02, 718.32it/s]
1795it [00:02, 728.54it/s]
1876it [00:03, 750.96it/s]
1958it [00:03, 769.62it/s]
2036it [00:03, 758.57it/s]
2113it [00:03, 733.22it/s]
2187it [00:03, 710.90it/s]
2259it [00:03, 688.83it/s]
2329it [00:03, 690.60it/s]
2399it [00:03, 691.90it/s]


In [46]:
assert len(nn_me_indices) == len(chosen_max)

In [47]:
nn_me_indices = index_process(nn_me_indices)


100%|██████████| 17972/17972 [00:00<00:00, 444641.25it/s]

In [123]:
nn_near_labels_reweights = []
nn_me_near_labels_max = np.sort(nn_me_near_labels)[-100]
for i in reweight_props:
    nn_near_labels_new = deepcopy(nn_near_labels)
    for jdx, j in enumerate(chosen_max[:int(i*len(nn_near_labels_new))]):
        if reweight_data[jdx] <= nn_me_near_labels_max:
        #if reweight_data[jdx]/nn_me_near_labels[nn_me_indices[jdx]] < 100:
            nn_near_labels_new[j] = nn_near_labels[j]*(reweight_data[jdx]/nn_me_near_labels[nn_me_indices[jdx]])
    nn_near_labels_reweights.append(nn_near_labels_new)

In [124]:
nn_xs_reweights = []
for i in nn_near_labels_reweights:
    new_xs = np.sum(np.concatenate((nn_cut_labels, i)))/np.sum(trials_nn)
    nn_xs_reweights.append(new_xs)
    print ("New XS is {}".format(new_xs))

New XS is 4.38744671886e-06
New XS is 4.11704632045e-06
New XS is 3.87949718042e-06
New XS is 3.65725826098e-06
New XS is 3.43824342958e-06
New XS is 3.26084784065e-06
